In [1]:
import pygame, sys, random
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
import math
import matplotlib.pyplot as plt
import pylab
import datetime

num_episodes = 3601

obstacleRadius = 12
agentRadius = 10

# get size of state and action from environment

boundaryPos = [10, 10]
boundaryLength = [70,70]
boundaryRadius = 40
initPosAgent = [boundaryPos[0] + boundaryLength[0] / 2, boundaryPos[1] + boundaryLength[1] / 2]
goalPos = [boundaryPos[0] + boundaryLength[0], boundaryPos[1] + boundaryLength[1]/2]
goalAngle = 0#random.randrange(0, 360) * math.pi / 180

moveObstacles = False
action_size = 9
obsNumber = 1
state_size = obsNumber * 2
# state_size = obsNumber + 1

Using TensorFlow backend.


In [2]:
# A2C(Advantage Actor-Critic) agent
class A2CAgent:
    def __init__(self, state_size, action_size):
        self.load_model = False
        
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        self.value_size = 1

        # These are hyper parameters for the Policy Gradient
        self.discount_factor = 0.99
        self.actor_lr = 0.00002
        self.critic_lr = 0.00005

        # create model for policy network
        self.actor = self.build_actor()
        self.critic = self.build_critic()

        if self.load_model:
            self.actor.load_weights("./Practice004_DataSave/Actor_Rev.h5")
            self.critic.load_weights("./Practice004_DataSave/Critic_Rev.h5")

    # approximate policy and value using Neural Network
    # actor: state is input and probability of each action is output of model
    def build_actor(self):
        actor = Sequential()
        actor.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        actor.add(Dense(self.action_size, activation='softmax', kernel_initializer='glorot_normal'))
        actor.summary()
        # See note regarding crossentropy in cartpole_reinforce.py
        actor.compile(loss='categorical_crossentropy', optimizer=Adam(lr=self.actor_lr))
        return actor

    # critic: state is input and value of state is output of model
    def build_critic(self):
        critic = Sequential()
        critic.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        critic.add(Dense(self.value_size, activation='linear', kernel_initializer='glorot_normal'))
        critic.summary()
        critic.compile(loss="mse", optimizer=Adam(lr=self.critic_lr))
        return critic

    # using the output of policy network, pick action stochastically
    def get_action(self, state):
        policy = self.actor.predict(state, batch_size=1).flatten()
#         print policy
        return np.random.choice(self.action_size, 1, p=policy)[0]

    # update policy network every episode
    def train_model(self, state, action, reward, next_state, done):
        target = np.zeros((1, self.value_size))
        advantages = np.zeros((1, self.action_size))

        value = self.critic.predict(state)[0]
        next_value = self.critic.predict(next_state)[0]

        if done:
            advantages[0][action] = reward - value
            target[0][0] = reward
        else:
            advantages[0][action] = reward + self.discount_factor * (next_value) - value
            target[0][0] = reward + self.discount_factor * next_value

        self.actor.fit(state, advantages, epochs=1, verbose=0)
        self.critic.fit(state, target, epochs=1, verbose=0)

In [3]:
def ckWall(xPos, yPos):
    if math.sqrt((xPos - initPosAgent[0]) ** 2 + (yPos - initPosAgent[1]) ** 2) > boundaryRadius - obstacleRadius:
        if xPos - initPosAgent[0] != 0:
            xPos = xPos - (xPos - initPosAgent[0])/abs(xPos - initPosAgent[0])
        if yPos - initPosAgent[1] != 0:
            yPos = yPos - (yPos - initPosAgent[1])/abs(yPos - initPosAgent[1])
    return [int(round(xPos)), int(round(yPos))]

In [4]:
def stateGenerator(obsPosition, agtPosition):
    returnSum = []
    for i in range(0,obsNumber):
        returnSum = returnSum + [agtPosition[0] - obsPosition[i][0], agtPosition[1] - obsPosition[i][1]]
#     returnSum = returnSum + [agtPosition[0] - boundaryRadius * math.cos(goalAngle), agtPosition[1] - boundaryRadius * math.sin(goalAngle)]
    returnSum = np.reshape(returnSum, [1, state_size])
    return returnSum

In [5]:
def takeAction(action):
    xAction = 0
    yAction = 0
    if action == 0:
        xAction = 1
    elif action == 1:
        xAction = 1
        yAction = 1
    elif action == 2:
        xAction = 1
        yAction = -1            
    elif action == 3:
        xAction = -1
        yAction = 1
    elif action == 4:
        xAction = -1
    elif action == 5:
        xAction = -1
        yAction = -1
    elif action == 6:
        yAction = -1
    elif action == 7:
        yAction = 1
    elif action == 8:
        xAction = 0
        yAction = 0
        
    return [xAction, yAction]

In [6]:
pygame.init()
screen = pygame.display.set_mode([100,100])
screen.fill([200, 200, 200])

# make A2C agent
agent = A2CAgent(state_size, action_size)

rList, episodes = [], []
obsAngleIdx = 0
# Make Obstacles (obsNumber)
obstaclePos = [[0, 0] for _ in range(obsNumber)]
for i in range(0,obsNumber):
#     obsRadius = random.randrange(agentRadius + obstacleRadius + 1, boundaryRadius)
    obsRadius = boundaryRadius
    obsAngle = obsAngleIdx * math.pi / 180
#     obsAngle = random.randrange(0,90) * math.pi / 180
    obstaclePos[i][0] = int(initPosAgent[0] + obsRadius * math.cos(obsAngle)) #boundaryPos[0] + random.randrange(1, boundaryLength[0])
    obstaclePos[i][1] = int(initPosAgent[1] + obsRadius * math.sin(obsAngle)) #boundaryPos[1] + random.randrange(1, boundaryLength[1])
circleFlag = False
initRandom = random.randrange(0,360)

for e in range(num_episodes):
    # Initialize
    done = False
    score = 0
    x = initPosAgent[0]
    y = initPosAgent[1]
    print("Episode ", e, "Starts!")
    state = stateGenerator(obstaclePos, [x,y])
    #state = np.reshape(state, [1, state_size])
    while not done:
        
        action = agent.get_action(state)
        
        xMove = 0
        yMove = 0
        
        [xMove, yMove] = takeAction(action)
        
        x = x + xMove
        y = y + yMove
        
        wallFlag = 0
        collisionFlag = 0
#         [x, y] = ckWall(x, y)
        pygame.draw.circle(screen, [100, 100, 255], [x,y], 10, 0)
        next_state = stateGenerator(obstaclePos, [x,y])

        if(math.sqrt((x - initPosAgent[0])**2 + (y - initPosAgent[1])**2) >= boundaryRadius - agentRadius):
            print("Goal Reached!")
            collisionFlag = 1
            done = True
        for i in range(0,obsNumber):
            if moveObstacles:
                obstaclePos[i][0] = obstaclePos[i][0] + random.randrange(1, 4) * random.randrange(-1,2)
                obstaclePos[i][1] = obstaclePos[i][1] + random.randrange(1, 4) * random.randrange(-1,2)
                [obstaclePos[i][0], obstaclePos[i][1]] = ckWall(obstaclePos[i][0], obstaclePos[i][1])
                
            pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], obstacleRadius, 0)
            if math.sqrt((x - obstaclePos[i][0])**2 + (y - obstaclePos[i][1])**2) <= obstacleRadius + agentRadius:
                print("Collision!")
                collisionFlag = -1
                done = True
        
        if not done:
            reward = 0
        else:
            if collisionFlag == 1:
                reward = -1000 #10000 * math.cos(math.atan2(y - initPosAgent[1], x - initPosAgent[0]))
                rList.append(1)
            elif collisionFlag == -1:
                reward = 1000
                rList.append(0)
        
        agent.train_model(state, action, reward, next_state, done)
        
        score += reward
        state = next_state
        if done:
            # every episode, plot the play time
            obstaclePos = [[0, 0] for _ in range(obsNumber)]
            for i in range(0,obsNumber):
#                 obsRadius = random.randrange(agentRadius + obstacleRadius + 1, boundaryRadius)
                obsRadius = boundaryRadius
                if obsAngleIdx >= 360:
                    circleFlag = True
                    initRandom = random.randrange(0,360)
                elif obsAngleIdx < 0:
                    circleFlag = False
                    initRandom = random.randrange(0,360)
                    
                if circleFlag == False:
                    obsAngleIdx += 1
                else:
                    obsAngleIdx -= 1
                
                obsAngle = (obsAngleIdx + initRandom) * math.pi / 180
                obstaclePos[i][0] = int(initPosAgent[0] + obsRadius * math.cos(obsAngle))
                obstaclePos[i][1] = int(initPosAgent[1] + obsRadius * math.sin(obsAngle))

            episodes.append(e)
#             pylab.plot(episodes, rList, 'b')
            
        #circle(Surface, color, pos, radius, width=0)
#         pygame.draw.circle(screen, [100,255,100], [goalPos[0],goalPos[1]], 10, 2)
        #rect(Surface, color, Rect, width=0)
#         pygame.draw.rect(screen, [255,100,100],[boundaryPos[0] - agentRadius, boundaryPos[1] - agentRadius, boundaryLength[0] + agentRadius * 2, boundaryLength[1] + agentRadius * 2],2)
        pygame.draw.circle(screen, [255,100,100], [initPosAgent[0], initPosAgent[1]], boundaryRadius, 2)
        pygame.display.flip()
        screen.fill([200,200,200])
    print(datetime.datetime.now().strftime('%H:%M:%S'), score)
    print "===================================================================================================="
    # save the model
    if e % 50 == 0:
        agent.actor.save_weights("./Practice004_DataSave/Actor_Rev.h5")
        agent.critic.save_weights("./Practice004_DataSave/Critic_Rev.h5")
#         pylab.savefig("./Practice004_DataSave/ActorCriticGraph.png")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               384       
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 1161      
Total params: 1,545
Trainable params: 1,545
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               384       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________
('Episode ', 0, 'Starts!')
Goal Reached!
('10:28:04', -1000)
('Episode ', 1,

Goal Reached!
('10:28:11', -1000)
('Episode ', 45, 'Starts!')
Goal Reached!
('10:28:12', -1000)
('Episode ', 46, 'Starts!')
Goal Reached!
('10:28:12', -1000)
('Episode ', 47, 'Starts!')
Goal Reached!
('10:28:12', -1000)
('Episode ', 48, 'Starts!')
Goal Reached!
('10:28:13', -1000)
('Episode ', 49, 'Starts!')
Goal Reached!
('10:28:13', -1000)
('Episode ', 50, 'Starts!')
Goal Reached!
('10:28:13', -1000)
('Episode ', 51, 'Starts!')
Goal Reached!
('10:28:14', -1000)
('Episode ', 52, 'Starts!')
Goal Reached!
('10:28:14', -1000)
('Episode ', 53, 'Starts!')
Goal Reached!
('10:28:14', -1000)
('Episode ', 54, 'Starts!')
Goal Reached!
('10:28:15', -1000)
('Episode ', 55, 'Starts!')
Goal Reached!
('10:28:15', -1000)
('Episode ', 56, 'Starts!')
Goal Reached!
('10:28:15', -1000)
('Episode ', 57, 'Starts!')
Goal Reached!
('10:28:16', -1000)
('Episode ', 58, 'Starts!')
Goal Reached!
('10:28:16', -1000)
('Episode ', 59, 'Starts!')
Goal Reached!
('10:28:16', -1000)
('Episode ', 60, 'Starts!')
Goal Rea

Collision!
('10:28:40', 1000)
('Episode ', 97, 'Starts!')
Collision!
('10:28:41', 1000)
('Episode ', 98, 'Starts!')
Collision!
('10:28:41', 1000)
('Episode ', 99, 'Starts!')
Collision!
('10:28:41', 1000)
('Episode ', 100, 'Starts!')
Collision!
('10:28:41', 1000)
('Episode ', 101, 'Starts!')
Collision!
('10:28:41', 1000)
('Episode ', 102, 'Starts!')
Collision!
('10:28:42', 1000)
('Episode ', 103, 'Starts!')
Collision!
('10:28:42', 1000)
('Episode ', 104, 'Starts!')
Collision!
('10:28:42', 1000)
('Episode ', 105, 'Starts!')
Collision!
('10:28:42', 1000)
('Episode ', 106, 'Starts!')
Collision!
('10:28:42', 1000)
('Episode ', 107, 'Starts!')
Collision!
('10:28:42', 1000)
('Episode ', 108, 'Starts!')
Collision!
('10:28:42', 1000)
('Episode ', 109, 'Starts!')
Collision!
('10:28:43', 1000)
('Episode ', 110, 'Starts!')
Collision!
('10:28:43', 1000)
('Episode ', 111, 'Starts!')
Collision!
('10:28:43', 1000)
('Episode ', 112, 'Starts!')
Collision!
('10:28:43', 1000)
('Episode ', 113, 'Starts!')


Collision!
('10:28:54', 1000)
('Episode ', 149, 'Starts!')
Collision!
('10:28:54', 1000)
('Episode ', 150, 'Starts!')
Collision!
('10:28:54', 1000)
('Episode ', 151, 'Starts!')
Collision!
('10:28:55', 1000)
('Episode ', 152, 'Starts!')
Collision!
('10:28:55', 1000)
('Episode ', 153, 'Starts!')
Collision!
('10:28:55', 1000)
('Episode ', 154, 'Starts!')
Collision!
('10:28:55', 1000)
('Episode ', 155, 'Starts!')
Collision!
('10:28:56', 1000)
('Episode ', 156, 'Starts!')
Collision!
('10:28:56', 1000)
('Episode ', 157, 'Starts!')
Collision!
('10:28:56', 1000)
('Episode ', 158, 'Starts!')
Collision!
('10:28:56', 1000)
('Episode ', 159, 'Starts!')
Collision!
('10:28:57', 1000)
('Episode ', 160, 'Starts!')
Collision!
('10:28:57', 1000)
('Episode ', 161, 'Starts!')
Collision!
('10:28:57', 1000)
('Episode ', 162, 'Starts!')
Collision!
('10:28:57', 1000)
('Episode ', 163, 'Starts!')
Collision!
('10:28:58', 1000)
('Episode ', 164, 'Starts!')
Collision!
('10:28:58', 1000)
('Episode ', 165, 'Starts!

Collision!
('10:29:06', 1000)
('Episode ', 201, 'Starts!')
Collision!
('10:29:06', 1000)
('Episode ', 202, 'Starts!')
Collision!
('10:29:06', 1000)
('Episode ', 203, 'Starts!')
Collision!
('10:29:06', 1000)
('Episode ', 204, 'Starts!')
Collision!
('10:29:06', 1000)
('Episode ', 205, 'Starts!')
Collision!
('10:29:07', 1000)
('Episode ', 206, 'Starts!')
Collision!
('10:29:07', 1000)
('Episode ', 207, 'Starts!')
Collision!
('10:29:07', 1000)
('Episode ', 208, 'Starts!')
Collision!
('10:29:07', 1000)
('Episode ', 209, 'Starts!')
Collision!
('10:29:08', 1000)
('Episode ', 210, 'Starts!')
Collision!
('10:29:08', 1000)
('Episode ', 211, 'Starts!')
Collision!
('10:29:08', 1000)
('Episode ', 212, 'Starts!')
Collision!
('10:29:08', 1000)
('Episode ', 213, 'Starts!')
Collision!
('10:29:08', 1000)
('Episode ', 214, 'Starts!')
Collision!
('10:29:09', 1000)
('Episode ', 215, 'Starts!')
Collision!
('10:29:09', 1000)
('Episode ', 216, 'Starts!')
Collision!
('10:29:09', 1000)
('Episode ', 217, 'Starts!

...

('10:35:19', 1000)
('Episode ', 3371, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3372, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3373, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3374, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3375, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3376, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3377, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3378, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3379, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3380, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3381, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3382, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3383, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3384, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3385, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3386, 'Starts!')
Collision!
('10:35:20', 1000)
('Episode ', 3387, 'S

Collision!
('10:35:23', 1000)
('Episode ', 3422, 'Starts!')
Collision!
('10:35:23', 1000)
('Episode ', 3423, 'Starts!')
Collision!
('10:35:23', 1000)
('Episode ', 3424, 'Starts!')
Collision!
('10:35:24', 1000)
('Episode ', 3425, 'Starts!')
Collision!
('10:35:24', 1000)
('Episode ', 3426, 'Starts!')
Collision!
('10:35:24', 1000)
('Episode ', 3427, 'Starts!')
Collision!
('10:35:24', 1000)
('Episode ', 3428, 'Starts!')
Collision!
('10:35:24', 1000)
('Episode ', 3429, 'Starts!')
Collision!
('10:35:24', 1000)
('Episode ', 3430, 'Starts!')
Collision!
('10:35:24', 1000)
('Episode ', 3431, 'Starts!')
Collision!
('10:35:25', 1000)
('Episode ', 3432, 'Starts!')
Collision!
('10:35:25', 1000)
('Episode ', 3433, 'Starts!')
Collision!
('10:35:25', 1000)
('Episode ', 3434, 'Starts!')
Collision!
('10:35:25', 1000)
('Episode ', 3435, 'Starts!')
Collision!
('10:35:25', 1000)
('Episode ', 3436, 'Starts!')
Collision!
('10:35:25', 1000)
('Episode ', 3437, 'Starts!')
Collision!
('10:35:26', 1000)
('Episode 

Collision!
('10:35:32', 1000)
('Episode ', 3473, 'Starts!')
Collision!
('10:35:32', 1000)
('Episode ', 3474, 'Starts!')
Collision!
('10:35:32', 1000)
('Episode ', 3475, 'Starts!')
Collision!
('10:35:33', 1000)
('Episode ', 3476, 'Starts!')
Collision!
('10:35:33', 1000)
('Episode ', 3477, 'Starts!')
Collision!
('10:35:33', 1000)
('Episode ', 3478, 'Starts!')
Collision!
('10:35:33', 1000)
('Episode ', 3479, 'Starts!')
Collision!
('10:35:33', 1000)
('Episode ', 3480, 'Starts!')
Collision!
('10:35:33', 1000)
('Episode ', 3481, 'Starts!')
Collision!
('10:35:34', 1000)
('Episode ', 3482, 'Starts!')
Collision!
('10:35:34', 1000)
('Episode ', 3483, 'Starts!')
Collision!
('10:35:34', 1000)
('Episode ', 3484, 'Starts!')
Collision!
('10:35:34', 1000)
('Episode ', 3485, 'Starts!')
Collision!
('10:35:34', 1000)
('Episode ', 3486, 'Starts!')
Collision!
('10:35:34', 1000)
('Episode ', 3487, 'Starts!')
Collision!
('10:35:35', 1000)
('Episode ', 3488, 'Starts!')
Collision!
('10:35:35', 1000)
('Episode 

Collision!
('10:35:38', 1000)
('Episode ', 3526, 'Starts!')
Collision!
('10:35:38', 1000)
('Episode ', 3527, 'Starts!')
Collision!
('10:35:39', 1000)
('Episode ', 3528, 'Starts!')
Collision!
('10:35:39', 1000)
('Episode ', 3529, 'Starts!')
Collision!
('10:35:39', 1000)
('Episode ', 3530, 'Starts!')
Collision!
('10:35:39', 1000)
('Episode ', 3531, 'Starts!')
Collision!
('10:35:39', 1000)
('Episode ', 3532, 'Starts!')
Collision!
('10:35:39', 1000)
('Episode ', 3533, 'Starts!')
Collision!
('10:35:39', 1000)
('Episode ', 3534, 'Starts!')
Collision!
('10:35:39', 1000)
('Episode ', 3535, 'Starts!')
Collision!
('10:35:39', 1000)
('Episode ', 3536, 'Starts!')
Collision!
('10:35:39', 1000)
('Episode ', 3537, 'Starts!')
Collision!
('10:35:39', 1000)
('Episode ', 3538, 'Starts!')
Collision!
('10:35:40', 1000)
('Episode ', 3539, 'Starts!')
Collision!
('10:35:40', 1000)
('Episode ', 3540, 'Starts!')
Collision!
('10:35:40', 1000)
('Episode ', 3541, 'Starts!')
Collision!
('10:35:40', 1000)
('Episode 

Collision!
('10:35:44', 1000)
('Episode ', 3577, 'Starts!')
Collision!
('10:35:44', 1000)
('Episode ', 3578, 'Starts!')
Collision!
('10:35:44', 1000)
('Episode ', 3579, 'Starts!')
Collision!
('10:35:44', 1000)
('Episode ', 3580, 'Starts!')
Collision!
('10:35:44', 1000)
('Episode ', 3581, 'Starts!')
Collision!
('10:35:44', 1000)
('Episode ', 3582, 'Starts!')
Collision!
('10:35:45', 1000)
('Episode ', 3583, 'Starts!')
Collision!
('10:35:45', 1000)
('Episode ', 3584, 'Starts!')
Collision!
('10:35:45', 1000)
('Episode ', 3585, 'Starts!')
Collision!
('10:35:45', 1000)
('Episode ', 3586, 'Starts!')
Collision!
('10:35:45', 1000)
('Episode ', 3587, 'Starts!')
Collision!
('10:35:45', 1000)
('Episode ', 3588, 'Starts!')
Collision!
('10:35:45', 1000)
('Episode ', 3589, 'Starts!')
Collision!
('10:35:45', 1000)
('Episode ', 3590, 'Starts!')
Collision!
('10:35:45', 1000)
('Episode ', 3591, 'Starts!')
Collision!
('10:35:46', 1000)
('Episode ', 3592, 'Starts!')
Collision!
('10:35:46', 1000)
('Episode 

In [ ]:
print("Percent of successful episodes: " + str(100.0 * sum(rList)/num_episodes) + "%")

# plt.bar(range(len(rList)), rList, color = "Blue", width = 0.00001)
# plt.show()